# Download ORACC

In [1]:
import os, sys, math
import requests
import json
import time
import random
import shutil
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
import oracc

In [3]:
root_out_dir = "/Volumes/FrankDisk"

In [4]:
out_dir = os.path.abspath(f"{root_out_dir}/oracc_zips")
os.makedirs(out_dir, exist_ok=True)
out_dir

'/Volumes/FrankDisk/oracc_zips'

## Get Project List

In [5]:
project_ids = oracc.get_project_ids()

In [6]:
len(project_ids)

136

In [7]:
random.shuffle(project_ids)
project_ids[:5]

['asbp/ninmed', 'saao/saa20', 'rinap/scores', 'caspo', 'blms']

## Download the projects

In [8]:
oracc.download_project_zip("aemw/amarna", out_dir)

http://build-oracc.museum.upenn.edu/json/aemw-amarna.zip


'/Volumes/FrankDisk/oracc_zips/aemw-amarna.zip'

In [9]:
for p in tqdm(project_ids):
    oracc.download_project_zip(p, out_dir, verbose=True)

  0%|          | 0/136 [00:00<?, ?it/s]

http://build-oracc.museum.upenn.edu/json/asbp-ninmed.zip
http://build-oracc.museum.upenn.edu/json/saao-saa20.zip
http://build-oracc.museum.upenn.edu/json/rinap-scores.zip
http://build-oracc.museum.upenn.edu/json/caspo.zip
http://build-oracc.museum.upenn.edu/json/blms.zip
http://build-oracc.museum.upenn.edu/json/saao-knpp.zip
http://build-oracc.museum.upenn.edu/json/atae-imgurenlil.zip
http://build-oracc.museum.upenn.edu/json/rimanum.zip
http://build-oracc.museum.upenn.edu/json/rimanum.zip does not exist.
http://build-oracc.museum.upenn.edu/json/glass.zip
http://build-oracc.museum.upenn.edu/json/ribo-sources.zip
http://build-oracc.museum.upenn.edu/json/atae-durkatlimmu.zip
http://build-oracc.museum.upenn.edu/json/contrib-amarna.zip
http://build-oracc.museum.upenn.edu/json/contrib-amarna.zip does not exist.
http://build-oracc.museum.upenn.edu/json/atae-guzana.zip
http://build-oracc.museum.upenn.edu/json/contrib-lambert.zip
http://build-oracc.museum.upenn.edu/json/contrib-lambert.zip does

http://build-oracc.museum.upenn.edu/json/akklove.zip does not exist.
http://build-oracc.museum.upenn.edu/json/hbtin.zip
http://build-oracc.museum.upenn.edu/json/saao-saa19.zip
http://build-oracc.museum.upenn.edu/json/saao-saa17.zip
http://build-oracc.museum.upenn.edu/json/ogsl.zip
http://build-oracc.museum.upenn.edu/json/atae.zip
http://build-oracc.museum.upenn.edu/json/cmawro-maqlu.zip
http://build-oracc.museum.upenn.edu/json/atae-marqasu.zip
http://build-oracc.museum.upenn.edu/json/dsst.zip
http://build-oracc.museum.upenn.edu/json/ribo-babylon8.zip
http://build-oracc.museum.upenn.edu/json/cmawro-cmawr2.zip
http://build-oracc.museum.upenn.edu/json/saao-saa09.zip
http://build-oracc.museum.upenn.edu/json/ckst.zip


In [10]:
tmp_dir = f"{out_dir}/tmp"
if os.path.exists(tmp_dir):
    os.rmdir(tmp_dir)